In [1]:
import math
from scipy.optimize import minimize
import numpy as np

In [2]:
### tools：

def minfunction(f, x0, bounds):
    """
    求一个函数在边界内的最小值
    f：函数
    x0：初始值
    bounds：边界条件。[(min, max),...]
    """
    res = minimize(f, x0, method='SLSQP', bounds=bounds)
    return res

def maxfunction(f, x0, bounds):
    """
    求一个函数在边界内的最大值
    f：函数
    x0：初始值
    bounds：边界条件。[(min, max),...]
    """
    fun = lambda X : -f(X)
    res = minimize(fun, x0, method='SLSQP', bounds=bounds)
    return res


In [3]:
### 目标函数：

bounds = [(13, 100), (0, 100)]
x_opt = [14.09500000000000064, 0.8429607892154795668]
y_opt = -6961.81387558015


def G06(X):
    y = (X[0] - 10) ** 3 + (X[1] - 20) ** 3

    g1 = -(X[0] - 5) ** 2 - (X[1] - 5) ** 2 + 100
    g2 = (X[0] - 6) ** 2 + (X[1] - 5) ** 2 - 82.81

    return y, g1, g2

In [4]:
# y = lambda X: G06(X)[0]
# g1 = lambda X: G06(X)[1]
# g2 = lambda X: G06(X)[2]
# x0 = [100,100]

In [5]:
# maxfunction(g2,x0,bounds)

In [6]:
### 适应度函数
def fitnessG06_num(X):
    """
    problem1关于数量惩罚的适应度函数
    X: 对应的解
    epsion：惩罚系数大于fmax-fmin的倍数
    """
    fmax = 1241000
    fmin = -7973
    epsion = 100
    count = np.sum(np.array(G06(X))[1:3]>0)
    
    return -(G06[0]+epsion*(fmax-fmin)*count)  

def fitnessG06_vio(X):
    """
    problem1关于数量惩罚的适应度函数
    X: 对应的解
    epsion：惩罚系数大于fmax-fmin的倍数
    """
    fmax = 1241000
    fmin = -7973
    vmax = 17814.19
    e = 0.01
    n = 2
    
    res = G06(X)
    fitness = -res[0]
    count = np.sum(np.array(res)[1:3]>0)
    
    if count > 0:
        for i in range(1,len(res)):
            if res[i] > 0:
                cathy = (fmax - fmin)*(e*vmax*n/count+res[i])/(e*vmax)
                fitness = fitness - cathy
    return fitness  

In [7]:
### PSO
class PSO_fit():
    def __init__(self, pN, dim, max_iter, bounds):
        self.wmax = 0.8 #最大的惯性权重
        self.wmin = 0.4 #最小的惯性权重
        self.c1 = 2  #加速因子
        self.c2 = 2  #加速因子
        self.pN = pN  # 粒子数量
        self.dim = dim  # 搜索维度
        self.max_iter = max_iter  # 迭代次数
        self.bounds = bounds
        
        self.X = np.zeros((self.pN, self.dim))  # 所有粒子的位置：一个pN*dim的矩阵
        self.V = np.zeros((self.pN, self.dim))  # 左右粒子的速度：一个pN*dim的矩阵
        self.pbest = np.zeros((self.pN, self.dim))  # 个体经历的最佳位置：一个pN*dim的矩阵
        self.gbest = np.zeros((self.pN, self.dim))      # 全局最佳位置：一个pN*dim的矩阵
        self.p_fit = np.full(self.pN,-100000000.0)  # 每个个体的历史最佳适应值
        self.g_fit = np.full(self.pN,-100000000.0)    # 全局最佳适应值
    
    def init_Population(self):
        """
        初始化种群
        :return:
        """
        
        for i in range(self.pN):
            for j in range(self.dim):
                self.X[i][j] = self.bounds[j][0] + (self.bounds[j][1] - self.bounds[j][0]) * np.random.random()
                self.V[i][j] = self.bounds[j][0] + (self.bounds[j][1] - self.bounds[j][0]) * np.random.random()
        for i in range(self.pN):
            tmp = fitnessG06_vio(self.X[i])
            if (tmp > self.p_fit[i]):
                self.p_fit[i] = tmp
                self.pbest[i] = self.X[i]
                
        for i in range(self.pN):
            indexes = np.random.randint(low=0,high=self.pN,size=int(self.pN/1500))
            for j in indexes:
                if self.p_fit[j] > self.g_fit[i]:
                    self.g_fit[i] = self.p_fit[j]
                    self.gbest[i] = self.pbest[j]

#             self.pbest[i] = self.X[i]
#             tmp = fitnessG01_num(self.X[i])
#             self.p_fit[i] = tmp
#             if (tmp > self.fit):
#                 self.fit = tmp
#                 self.gbest = self.X[i]
        
    def iterator(self):
        """
        迭代
        """
        for t in range(self.max_iter):
#             w = self.wmin + (self.wmax - self.wmin) * np.exp(- (4 * t / self.max_iter) * (4 * t / self.max_iter))
            w = 0.85
            for i in range(self.pN):
                self.V[i] = w*self.V[i]+self.c1*np.random.random()*(self.pbest[i]-self.X[i])+self.c2*np.random.random()*(self.gbest[i]-self.X[i])
                self.X[i] = self.X[i] + self.V[i]
                
                """
                控制在搜索范围内
                """ 
                for j in range(self.dim):
                    if self.X[i][j] < self.bounds[j][0]:
                        self.X[i][j] = self.bounds[j][0]
                    if self.X[i][j] > self.bounds[j][1]:
                        self.X[i][j] = self.bounds[j][1]
                        
            for i in range(self.pN):
                tmp = fitnessG06_vio(self.X[i])
                if (tmp > self.p_fit[i]):
                    self.p_fit[i] = tmp
                    self.pbest[i] = self.X[i]
            for i in range(self.pN):
                indexes = np.random.randint(low=0,high=self.pN,size=int(self.pN/1500))
                for j in indexes:
                    if self.p_fit[j] > self.g_fit[i]:
                        self.g_fit[i] = self.p_fit[j]
                        self.gbest[i] = self.pbest[j]
            print(self.g_fit)  
        return self.gbest, self.g_fit

In [8]:
PSO = PSO_fit(pN=15000, dim=2, max_iter=500, bounds=bounds)

In [9]:
PSO.init_Population()

In [ ]:
gbest, gfit = PSO.iterator()

[-8596052.89611244 -2605603.9141313  -3353527.02192373 ...
 -4608491.93051937 -2567095.24355977 -3417358.74067274]
[-2567095.24355977 -2567095.24355977 -2567095.24355977 ...
 -2567095.24355977 -2567095.24355977 -2567095.24355977]
[-2567095.24355977 -2567095.24355977 -2527483.00857784 ...
 -2567095.24355977 -2567095.24355977 -2567095.24355977]
[-2567095.24355977 -2567095.24355977 -2527483.00857784 ...
 -2567095.24355977 -2567095.24355977 -2567095.24355977]
[-2567095.24355977 -2494901.41030942 -2527483.00857784 ...
 -2567095.24355977 -2567095.24355977 -2509953.67169981]
[-2567095.24355977 -2494901.41030942 -2519103.53863131 ...
 -2567095.24355977 -2567095.24355977 -2509953.67169981]
[-2567095.24355977 -2494901.41030942 -2519103.53863131 ...
 -2566247.30163863 -2512588.50520304 -2509953.67169981]
[-2520483.91111069 -2494901.41030942 -2519103.53863131 ...
 -2566247.30163863 -2512588.50520304 -2509953.67169981]
[-2520483.91111069 -2494901.41030942 -2519103.53863131 ...
 -2566247.30163863 -2

[6935.31339271 6942.04244068 6872.26148068 ... 6928.02527267 6897.99374893
 6887.84282779]
[6935.31339271 6942.04244068 6872.26148068 ... 6928.02527267 6897.99374893
 6887.84282779]
[6935.31339271 6942.04244068 6893.27570453 ... 6928.02527267 6913.57583725
 6887.84282779]
[6935.31339271 6942.04244068 6893.27570453 ... 6928.02527267 6913.57583725
 6888.00026989]
[6935.31339271 6942.04244068 6893.27570453 ... 6928.02527267 6913.57583725
 6888.00026989]
[6935.31339271 6942.04244068 6893.27570453 ... 6928.02527267 6913.57583725
 6889.70811028]
[6935.31339271 6942.04244068 6893.27570453 ... 6928.02527267 6913.57583725
 6889.70811028]
[6935.31339271 6942.04244068 6893.27570453 ... 6928.02527267 6913.57583725
 6889.70811028]
[6935.31339271 6942.04244068 6893.27570453 ... 6928.02527267 6913.57583725
 6889.70811028]
[6935.31339271 6942.04244068 6893.27570453 ... 6928.02527267 6913.57583725
 6904.28911124]
[6935.31339271 6942.04244068 6893.27570453 ... 6928.02527267 6913.57583725
 6904.28911124]

[6958.38729422 6955.81866813 6959.90436804 ... 6961.49124993 6952.38499401
 6955.25962864]
[6959.43285094 6955.81866813 6959.90436804 ... 6961.49124993 6952.38499401
 6955.25962864]
[6959.43285094 6955.81866813 6959.90436804 ... 6961.49124993 6952.38499401
 6955.25962864]
[6959.43285094 6955.81866813 6959.90436804 ... 6961.49124993 6952.38499401
 6955.25962864]
[6959.43285094 6955.81866813 6959.90436804 ... 6961.49124993 6952.38499401
 6955.25962864]
[6959.43285094 6955.81866813 6959.90436804 ... 6961.49124993 6952.38499401
 6955.25962864]
[6959.43285094 6955.81866813 6959.90436804 ... 6961.49124993 6953.30732877
 6955.25962864]
[6959.43285094 6955.81866813 6959.90436804 ... 6961.49124993 6953.30732877
 6955.25962864]
[6959.43285094 6955.81866813 6959.90436804 ... 6961.49124993 6953.30732877
 6955.25962864]
[6959.43285094 6955.81866813 6959.90436804 ... 6961.49124993 6953.30732877
 6955.25962864]
[6959.43285094 6955.81866813 6959.90436804 ... 6961.49124993 6953.30732877
 6955.25962864]

[6961.35634402 6960.81266525 6961.59543139 ... 6961.49124993 6960.52617668
 6961.35962512]
[6961.35634402 6960.81266525 6961.59543139 ... 6961.49124993 6960.80633343
 6961.35962512]
[6961.35634402 6960.81266525 6961.59543139 ... 6961.49124993 6961.03332462
 6961.35962512]
[6961.35634402 6960.81266525 6961.59543139 ... 6961.49124993 6961.03332462
 6961.35962512]
[6961.35634402 6960.81266525 6961.59543139 ... 6961.49124993 6961.03332462
 6961.35962512]
[6961.35634402 6960.81266525 6961.59543139 ... 6961.49124993 6961.03332462
 6961.35962512]
[6961.35634402 6960.81266525 6961.59543139 ... 6961.49124993 6961.03332462
 6961.35962512]
[6961.35634402 6960.81266525 6961.59543139 ... 6961.49124993 6961.03332462
 6961.35962512]
[6961.35634402 6960.81266525 6961.59543139 ... 6961.49124993 6961.03332462
 6961.35962512]
[6961.35634402 6960.81266525 6961.59543139 ... 6961.49124993 6961.03332462
 6961.35962512]
[6961.35634402 6960.81266525 6961.59543139 ... 6961.49124993 6961.03332462
 6961.35962512]

[6961.7623328  6961.65589743 6961.6880382  ... 6961.73236662 6961.64307991
 6961.78810649]
[6961.7623328  6961.65589743 6961.6880382  ... 6961.73236662 6961.64307991
 6961.78810649]
[6961.7623328  6961.65589743 6961.6880382  ... 6961.73236662 6961.64307991
 6961.78810649]
[6961.7623328  6961.65589743 6961.6880382  ... 6961.73236662 6961.64307991
 6961.78810649]
[6961.7623328  6961.65589743 6961.6880382  ... 6961.73236662 6961.64307991
 6961.78810649]
[6961.7623328  6961.65589743 6961.6880382  ... 6961.73236662 6961.64307991
 6961.78810649]
[6961.7623328  6961.65589743 6961.6880382  ... 6961.73236662 6961.64307991
 6961.78810649]
[6961.7623328  6961.65589743 6961.6880382  ... 6961.73236662 6961.64307991
 6961.78810649]
[6961.7623328  6961.65589743 6961.6880382  ... 6961.73236662 6961.64307991
 6961.78810649]
[6961.7623328  6961.65589743 6961.6880382  ... 6961.73236662 6961.64307991
 6961.78810649]
[6961.7623328  6961.65589743 6961.6880382  ... 6961.73236662 6961.64307991
 6961.78810649]

In [ ]:
G06(gbest[0])

In [ ]:
gbest[0]